In [1]:
from manim import *
import numpy as np

config.media_width = "75%"
config.verbosity = "WARNING"
#%%manim -qm NuclearScatteringBasic

# Physical quantity colors
RADIUS_COLOR = YELLOW
MOMENTUM_COLOR = GREEN  # Bright green
ANGULAR_MOMENTUM_COLOR = PURPLE  # Medium purple
SPIN_UP_COLOR = RED  # Bright red
SPIN_DOWN_COLOR = BLUE  # Bright blue
FORCE_COLOR = ORANGE
SPIN_RIGHT_COLOUR = DARK_BROWN
SPIN_LEFT_COLOUR = TEAL_B
NUCLEUS_COLOR = GREY

In [3]:
%%manim -qh SpinOrbitDeflection2

class SpinOrbitDeflection2(ThreeDScene):
    def setup_scene(self):
        # Set up the camera
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes and grid
        axes = ThreeDAxes()
        grid = NumberPlane()
        axes.set_opacity(0.2)
        grid.set_opacity(0.2)
        
        # Add nucleus
        nucleus = Sphere(radius=0.2, color=NUCLEUS_COLOR).move_to(ORIGIN)
        nucleus.set_fill(color=NUCLEUS_COLOR, opacity=0.8)
        
        # Add nucleus label
        nucleus_label = Text("Target\nNucleus", font_size=24)
        nucleus_label.move_to(nucleus.get_center() + RIGHT + DOWN)

        self.add(grid, nucleus)
        
        # Create legend
        legend_group = VGroup()
        
        # Spin up dot and label
        spin_up_dot = Dot(color=SPIN_UP_COLOR)
        spin_up_text = Text("Spin Up", font_size=24, color=SPIN_UP_COLOR).next_to(spin_up_dot, RIGHT)
        spin_up_group = VGroup(spin_up_dot, spin_up_text)
        
        # Spin down dot and label
        spin_down_dot = Dot(color=SPIN_DOWN_COLOR)
        spin_down_text = Text("Spin Down", font_size=24, color=SPIN_DOWN_COLOR)
        spin_down_text.next_to(spin_down_dot, RIGHT)
        spin_down_group = VGroup(spin_down_dot, spin_down_text)
        
        # Arrange legend items vertically
        legend_group.add(spin_up_group, spin_down_group)
        legend_group.arrange(DOWN, aligned_edge=LEFT, buff=0.3)
        
        # Add background rectangle
        background = SurroundingRectangle(legend_group, color=WHITE, fill_opacity=0.1, buff=0.2)
        legend_group.add_to_back(background)
        
        # Position in top right
        legend_group.to_corner(UR, buff=0.5)
        
        # Add legend to the scene fixed to the frame
        self.add_fixed_in_frame_mobjects(legend_group)
        
        self.wait(1)
        
        # Save nucleus and grid for later use
        self.nucleus = nucleus
        self.grid = grid

    def create_particle(self, start_point, end_point, color):
        control_point = np.array([0, start_point[1], 0]) + np.array([2, 0, 0])
        particle = Sphere(radius=0.1, color=color).move_to(start_point)
        particle.set_fill(color=color, opacity=0.8)
        trace = TracedPath(particle.get_center, stroke_color=color)
        self.add(trace)
        return particle, control_point, end_point

    def animate_particle(self, particle, control_point, end_point):
        # Combine straight and curved motion into one animation
        self.play(FadeIn(particle))
        
        start_y = particle.get_center()[1]
        
        def combined_path(t):
            if t < 0.333:  # First third of animation is straight motion
                # Normalize t to [0,1] for this segment
                t1 = t * 3
                return np.array([-2 + t1 * 2, start_y, 0])
            else:  # Remaining time is bezier curve
                # Normalize t to [0,1] for this segment
                t2 = (t - 0.333) * 1.5
                s = np.array([0, start_y, 0])
                return (1-t2)**2 * s + 2*(1-t2)*t2 * control_point + t2**2 * end_point
        
        self.play(
            UpdateFromAlphaFunc(
                particle,
                lambda mob, alpha: mob.move_to(combined_path(alpha)),
                run_time=1.5,
                rate_func=smooth
            )
        )

    def construct(self):
        self.setup_scene()

        # Create and animate spin +1 particles (red)
        particle1, cp1, ep1 = self.create_particle(
            np.array([-2, -1, 0]), 
            np.array([4, 3, 0]), 
            RED
        )
        particle2, cp2, ep2 = self.create_particle(
            np.array([-2, 1, 0]), 
            np.array([3.5, 5.5, 0]), 
            RED
        )

        # Create and animate spin -1 particles (blue)
        particle3, cp3, ep3 = self.create_particle(
            np.array([-2, -1, 0]), 
            np.array([4, -5, 0]), 
            BLUE
        )
        particle4, cp4, ep4 = self.create_particle(
            np.array([-2, 1, 0]), 
            np.array([4, -3, 0]), 
            BLUE
        )

        # Animate all particles
        for particle, cp, ep in [
            (particle1, cp1, ep1),
            (particle2, cp2, ep2),
            (particle3, cp3, ep3),
            (particle4, cp4, ep4)
        ]:
            self.animate_particle(particle, cp, ep)

Manim Community v0.18.1

In [6]:
%%manim -qh NucleonForce

class NucleonForce(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create nucleon at center
        nucleon = Sphere(radius=0.3, color=GRAY)
        
        # Create main coordinate axes
        axes = ThreeDAxes(
            x_range=[-2, 2, 1],
            y_range=[-2, 2, 1],
            z_range=[-2, 2, 1],
            axis_config={"include_tip": True, "tip_width": 0.2, "tip_height": 0.2}
        ).set_opacity(0.8)

        # Add labels with nuclear physics convention
        z_label = MathTex("z").next_to(axes.z_axis.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        y_label = MathTex("y").next_to(axes.y_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        x_label = MathTex("x").next_to(axes.x_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Create momentum equations (fixed in frame)
        initial_momentum = MathTex(
            "\\vec{p} = (p_0, 0, 0)",
            color=WHITE
        ).to_corner(UL).shift(0.1*RIGHT + 0.1*DOWN).set_opacity(0)
        self.add_fixed_in_frame_mobjects(initial_momentum)

        final_momentum = MathTex(
            "\\vec{p}\\,' = (p' \\sin", "\\vartheta", "\\cos", " \\phi,", 
            "p' \\sin"," \\vartheta","\\sin ",",\\phi,",
            "p' \\cos",  "\\vartheta)",
            color=WHITE
        ).to_corner(DL).shift(0.1 * RIGHT + 0.1* UP).set_opacity(0)
    
        self.add_fixed_in_frame_mobjects(final_momentum)
        final_momentum[1].set_color(BLUE)
        final_momentum[3].set_color(GREEN)
        final_momentum[5].set_color(BLUE)
        final_momentum[7].set_color(GREEN)
        final_momentum[9].set_color(BLUE)

        # Create phi arc (azimuthal angle)
        phi_arc = always_redraw(
            lambda: Arc(
                radius=2,
                angle=2 * PI,
                color=GREEN
            ).rotate(90 * DEGREES, axis=UP)
        )

        # Create theta arc (polar angle)
        theta_arc = always_redraw(
            lambda: Arc(
                radius=1,
                angle=1/2 *PI,
                color=BLUE
            ).rotate(90 * DEGREES, axis=RIGHT).move_to([0.5, 0,0.5])
        )

        # Add angle labels
        theta_label = MathTex("\\theta").move_to([0.9, 0, 0.9]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        theta_label.color = BLUE
        phi_label = MathTex("\\phi").move_to([0, 1.6, 1.6]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        phi_label.color = GREEN

        # Create incoming nucleon (small red sphere)
        incoming_nucleon = Sphere(radius=0.1, color=RED)
        incoming_nucleon.move_to([-2, 0, 0])  # Start from negative x-axis

        # Create the scattering path
        incoming_path = Line(start=[-2, 0, 0], end=[0, 0, 0], color=RED)
        
        # Create scattered path (at some angle)
        scattered_path = Line(
            start=[0, 0, 0],
            end=[1, 1, -1],  # Example scattering direction
            color=RED
        )

        # Add everything to the scene
        self.add(axes, y_label, z_label, x_label)
        self.play(Create(theta_arc), Write(theta_label), run_time=3)
        self.play(Write(phi_label), Create(phi_arc), run_time=3)
    

        # Add scattering animation
        self.play(Create(incoming_path))
        self.play(
            incoming_nucleon.animate.move_to([-2, 0, 0]),
            run_time=0.1
        )
        self.play(
            MoveAlongPath(incoming_nucleon, incoming_path),
            Write(initial_momentum),
            initial_momentum.animate.set_opacity(1),
            run_time=2

        )
        self.play(
            Create(scattered_path),
            MoveAlongPath(incoming_nucleon, scattered_path),
            Write(final_momentum),
            final_momentum.animate.set_opacity(1),
            run_time=2
        )

        # rotate the camera to see the scattering
        self.begin_ambient_camera_rotation(rate=0.3)
        self.wait(3)
        self.stop_ambient_camera_rotation()

        self.wait(3)

Manim Community v0.18.1

In [ ]:
%%manim -qh NuclearScatteringAsym6

class NuclearScatteringAsym6(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)

        # Create nucleus
        nucleus = Sphere(radius=0.2, color=GRAY)

        # Create particle at (0, -1, 0)
        particle = Sphere(radius=0.1, color=RED)
        particle.move_to([0, -1, 0])

        # Orbit path in the YZ plane
        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)

        # Radial force vector (scaled r-vector)
        radial_force = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=PURPLE,
            buff=0
        )

        # p × S vector (constant in Y direction)
        cross_product = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, 1, 0]),
            color=ORANGE,
            buff=0
        )

        # Updater for radial force vector
        def update_radial_force(vector):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            
            S_cross_p = np.array([0, 1, 0])  # This is p × S
            f_radial = np.dot(r, S_cross_p)
            
            force_vector = f_radial * r_hat
            vector.put_start_and_end_on(ORIGIN, force_vector)

        # Updater for cross product vector
        def update_cross_product(vector):
            pos = particle.get_center()
            vector.put_start_and_end_on(pos, pos + np.array([0, 1, 0]))

        radial_force.add_updater(update_radial_force)
        cross_product.add_updater(update_cross_product)

        # Add everything to scene
        self.add(nucleus, particle, orbit_path, radial_force, cross_product, axes)

        # Create the fixed label on top
        label_text = MathTex("F_{radial} = ", "\\vec{r}", "\\cdot", 
            "\\big(", "\\vec{p}", "\\times", "\\vec{S}", "\\big) = ", color=WHITE)
        label_text.to_corner(UR).shift(LEFT * 2)
        self.add_fixed_in_frame_mobjects(label_text)
        label_text[0].set_color(FORCE_COLOR)  # F
        label_text[1].set_color(YELLOW)  # r
        label_text[4].set_color(MOMENTUM_COLOR)    # p
        label_text[5].set_color(SPIN_UP_COLOR)  # S

        # Add cross product label
        cross_label = MathTex("F_{grad} = ","\\vec{p}", "\\times", "\\vec{S}"," = 1" ,  color=WHITE)
        cross_label.to_corner(UR).shift(LEFT * 2 + DOWN * 0.8)
        self.add_fixed_in_frame_mobjects(cross_label)
        cross_label[0].set_color(FORCE_COLOR)  # F
        cross_label[1].set_color(MOMENTUM_COLOR)    #
        cross_label[3].set_color(SPIN_UP_COLOR) # S


        # Create the updating value text
        display_text = MathTex("00.00", color=WHITE)
        display_text.next_to(label_text, RIGHT, buff=0.32)
        self.add_fixed_in_frame_mobjects(display_text)

        # Update just the value text
        def update_text(mob):
            r = particle.get_center()
            S_cross_p = np.array([0, 1, 0])
            f_radial = np.dot(r, S_cross_p)
            new_text = MathTex(f"{f_radial:.2f}", color=WHITE)
            new_text.move_to(mob)
            mob.become(new_text)

        display_text.add_updater(update_text)

        # # Begin ambient camera rotation
        # self.begin_ambient_camera_rotation(rate=0.9)
        
        # Create one long rotation animation
        self.play(
            Rotating(
                particle,
                about_point=ORIGIN,
                axis=RIGHT,
                angle=TAU,  # Rotate 3 full times
                run_time=4,  # Total time for all rotations
                rate_func=linear
            )
        )

        self.wait(2)

        # Clean up updaters
        radial_force.clear_updaters()
        cross_product.clear_updaters()
        display_text.clear_updaters()

In [5]:
%%manim -qh SpinScatteringAnimation

class SpinScatteringAnimation(Scene):
    def construct(self):
        # Initial formulas for p and p' with vector notation
        initial_p = MathTex(
            r"\vec{p} = (0, 0, p)"
        ).shift(UP * 2)
        
        initial_p_prime = MathTex(
            r"\vec{p}^{\,\prime} = (p'\sin\vartheta\cos\phi,",
            r"p'\sin\vartheta\sin\phi,",
            r"p'\cos\vartheta)"
        ).next_to(initial_p, DOWN)

        # Create first scene with initial formulas
        self.play(Write(initial_p))
        self.play(Write(initial_p_prime))
        self.wait(2)

        # Azimuthal distribution formula
        azimuthal_title = Text("Angular Distribution of scattered nucleons:", font_size=36).shift(UP * 2.2)
        azimuthal_formula1 = MathTex(
            r"\cos(\phi - \phi_0) =", # index 0
            r"\cos\phi\cos\phi_0 + \sin\phi\sin\phi_0" # index 1
        ).move_to([0, -2, 0])
        
        # Start with right side hidden
        azimuthal_formula1[1].set_opacity(0)

        # Animate transition to azimuthal distribution
        self.play(
            FadeOut(initial_p),
            initial_p_prime.animate.scale(0.8).shift(UP * 2),
            FadeIn(azimuthal_title),
        )
        self.wait(1)

        # Add cross product term with vector notation
        cross_product = MathTex(
            r"-\vec{p} \times \vec{S} = (pS_y, -pS_x, 0)"
        ).move_to([0, 0.5, 0])

        # Show cross product and its relationship
        self.play(Write(cross_product))
        self.wait(2)

        # Final preferred angle formulas with step-by-step transformation
        preferred_angle = MathTex(
            r"\tan\phi_0 ", 
            r"= -\frac{S_x}{S_y}"
        ).move_to([0, 0, 0])

        preferred_angle2 = MathTex(
            r"\frac{\sin\phi_0}{\cos\phi_0} ",
            r"= -\frac{S_x}{S_y}"
        ).move_to([0, 0, 0])

        self.wait(2)

        # Show initial form of preferred angle
        self.play(
            Write(preferred_angle),
            cross_product.animate.shift(UP * 0.5)
        )
        self.wait(2)

        # Reveal the right side of the equation with trigonometric identity
        self.play(Write(azimuthal_formula1[0]))
        self.wait(2)
        self.play(
            azimuthal_formula1[1].animate.set_opacity(1),
            run_time=2
        )
        self.wait(2)

        # Proportional relationship with spin components
        azimuthal_formula2 = MathTex(
            r"\cos(\phi - \phi_0) ", # index 0
            r"\propto S_y\cos\phi - S_x\sin\phi" # index 1
        ).move_to([0, -2, 0])

        self.play(Transform(preferred_angle, preferred_angle2))

        # Final transition
        self.play(Transform(azimuthal_formula1, azimuthal_formula2))
        self.wait(2)

        self.play(FadeOut(preferred_angle), FadeOut(cross_product), FadeOut(initial_p_prime),
            azimuthal_formula1.animate.move_to([0, 0, 0]))
        self.wait(5)
        
        # Final distribution formula with color coding
        final_formula = MathTex(
            r"f(p, \vartheta, \phi)", # 0
            r"=", # 1
            r"\frac{\varepsilon(p, \vartheta)}{2\pi}", # 2
            r"(1", # 3
            r"+", # 4
            r"A_y(p, \vartheta)", # 5
            r"S_y", # 6
            r"\cos\phi", # 7
            r"-", # 8
            r"A_y(p, \vartheta)", # 9
            r"S_x", # 10
            r"\sin\phi", # 11
            r")" # 12
        ).move_to([0, 0, 0])

        # Color scheme
        final_formula[0].set_color(WHITE)        # function name
        final_formula[2].set_color(BLUE)         # phase space factor
        final_formula[5].set_color(GREEN)        # first Ay
        final_formula[9].set_color(GREEN)        # second Ay
        final_formula[6].set_color(PURPLE)       # Sy term
        final_formula[10].set_color(RED)         # Sx term
        final_formula[7].set_color(YELLOW)       # cos phi
        final_formula[11].set_color(YELLOW)      # sin phi

        # Write the full formula first
        self.play(Transform(azimuthal_formula1, final_formula))
        self.wait(4)

        # Define each explanation step
        steps = [
            #(final_formula[0], "Angular Distribution"),
            (final_formula[2], "Polarimeter Efficiency"),
            (final_formula[3], "Unpolarized Distribution"),
            #([final_formula[5], final_formula[9]], "Analyzing Power"),
            ([final_formula[5], final_formula[6], final_formula[7]], "Up-Down Asymmetry"),
            ([final_formula[8], final_formula[9], final_formula[10], final_formula[11]], "Left-Right Asymmetry")
        ]

        # Step through each explanation
        current_brace = None
        current_text = None

        for parts, explanation in steps:
            if isinstance(parts, list):
                new_brace = Brace(VGroup(*parts), DOWN)
            else:
                new_brace = Brace(parts, DOWN)
            
            new_text = Text(explanation, font_size=24).next_to(new_brace, DOWN)

            if current_brace is None:
                self.play(
                    Create(new_brace),
                    Write(new_text)
                )
            else:
                self.play(
                    ReplacementTransform(current_brace, new_brace),
                    ReplacementTransform(current_text, new_text)
                )

            current_brace = new_brace
            current_text = new_text
            self.wait(1.5)

        # Final fade out
        self.play(
            FadeOut(current_brace),
            FadeOut(current_text)
        )
        self.wait(1)

Manim Community v0.18.1

In [9]:
%%manim -qh NucleonScatteringComplete

from manim import *
import numpy as np

class NucleonScatteringComplete(ThreeDScene):
    def construct(self):
        # Common setup
        self.set_camera_orientation(phi=75 * DEGREES, theta=130 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        self.add(analyzer)
        
        # Function for single scattering event
        def create_scattering_event(start_y, angle_deg, y_angle_deg, run_time_factor):
            nucleon = Sphere(radius=0.1, fill_opacity=0.8).set_color(RED)
            nucleon.move_to([-3, start_y, 0])
            
            # Calculate end point based on both angles
            xz_angle_rad = angle_deg * DEGREES
            y_angle_rad = y_angle_deg * DEGREES
            end_x = 2
            path_length = np.sqrt(end_x * end_x + start_y * start_y)
            end_y = start_y + path_length * np.tan(xz_angle_rad)
            end_z = path_length * np.tan(y_angle_rad)
            
            path1 = Line([-3, start_y, 0], [0, start_y, 0])
            path2 = Line([0, start_y, 0], [end_x, end_y, end_z])
            
            # Add reference line
            ref_line = DashedLine([0, start_y, 0], [end_x, start_y, 0], color=RED_A)
            
            # Add nucleon
            self.add(nucleon)
            
            # Animate movement
            self.play(
                MoveAlongPath(nucleon, path1),
                rate_func=linear,
                run_time=2 * run_time_factor
            )
            
            self.wait(0.1 * run_time_factor)
            
            self.play(
                MoveAlongPath(nucleon, path2),
                rate_func=linear,
                run_time=1.5 * run_time_factor
            )
            
            # Draw trajectory and reference line
            trajectory = VGroup(
                DashedLine([-3, start_y, 0], [0, start_y, 0]),
                DashedLine([0, start_y, 0], [end_x, end_y, end_z])
            ).set_color(GREY_C)
            
            self.play(
                Create(trajectory),
                Create(ref_line),
                run_time=0.5 * run_time_factor
            )
            self.wait(0.3 * run_time_factor)
            
            # Remove everything
            self.play(
                FadeOut(nucleon),
                FadeOut(trajectory),
                FadeOut(ref_line),
                run_time=0.3 * run_time_factor
            )

        # [Rest of the code remains the same...]
        def create_enhanced_fan():
            fan = VGroup()
            n_lines = 35
            
            for i in range(n_lines):
                phi = i * 2 * PI / n_lines + np.random.uniform(-0.1, 0.1)
                angle = np.random.uniform(5, 17) * DEGREES
                distance = np.random.uniform(2.2, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE
                ).set_opacity(0.4)
                fan.add(line)
            
            for _ in range(15):
                phi = np.random.uniform(0, 2*PI)
                angle = np.random.uniform(3, 18) * DEGREES
                distance = np.random.uniform(1.5, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE_D
                ).set_opacity(0.3)
                fan.add(line)
            
            return fan

        # Show individual scattering events
        scatter_events = [
            (0, 14, 5, 0.9),
            (0.2, -10, -8, 0.7),
            (0.1, 18, 3, 0.5),
            (0.4, -25, 7, 0.3),
            (0.3, 22, -10, 0.3)
        ]
        
        for start_y, angle, y_angle, time_factor in scatter_events:
            create_scattering_event(start_y, angle, y_angle, time_factor)
        
        # Create and show final distribution
        title = Text("Azimuthal Distribution", font_size=24).to_edge(UP)
        self.add_fixed_in_frame_mobjects(title)
        
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        reference_line = DashedLine([0, 0, 0], [2.5, 0, 0], color=RED_A)
        fan = create_enhanced_fan()
        cone = Cone(
            direction=[-1, 0, 0],
            height=2.5,
            base_radius=0.7,    
            color=BLUE
        ).set_opacity(0.5)
        
        # Show distribution
        self.play(
            Create(incoming_line),
            Create(fan, lag_ratio=0.01),
            Create(reference_line),
            run_time=1.5
        )
        self.play(Create(cone))
        self.wait(2)
        
        # Stop rotation and hold final frame
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1

In [6]:
%%manim -pqh NuclearScatteringVectors3D_final

class NuclearScatteringVectors3D_final(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)
        
        # Create axes with labels
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        
        # Create a xy plane grid for reference
        grid = NumberPlane()
        grid.set_opacity(0.2)
        
        # Create nucleus (sphere) with better visibility
        nucleus = Sphere(radius=0.2, color=NUCLEUS_COLOR)
        nucleus.set_fill(color=NUCLEUS_COLOR, opacity=0.8)
        
        # Create and position the nucleus label better
        nucleus_label = Text("Target\nNucleus", font_size=24)
        
        # Incident particles paths
        start_point1 = np.array([-3, -1, 0])
        end_point1 = np.array([0, -1, 0])
        
        # Create particle with trail
        def create_particle_with_trail(color, start_point):
            particle = Sphere(radius=0.1, color=color)
            particle.set_fill(color=color, opacity=1)
            particle.move_to(start_point)
            return particle
            
        particle1 = create_particle_with_trail(RED, start_point1)
        
        # Create the trace
        trace = TracedPath(particle1.get_center, stroke_width=2, stroke_color=RED, 
                          stroke_opacity=0.8, dissipating_time=10)

        # Add spin vectors with better visibility
        def create_spin_arrow(point, direction=1):
            return Arrow3D(
                start=point + np.array([0, 0, -0.2]),
                end=point + np.array([0, 0, 0.3]),
                color=SPIN_UP_COLOR if direction == 1 else SPIN_DOWN_COLOR,
                thickness=0.01,
                height=0.1,
                base_radius=0.05,
            )
            
        spin1 = create_spin_arrow(start_point1)
        
        # Initial scene setup
        self.play(Create(nucleus))
        self.wait(2)
        self.play(Create(grid))
        
        # Add trace before particle animation
        self.add(trace)
        
        # Particle approach animation
        self.play(FadeIn(particle1))
        self.wait(0.5)
        self.play(Create(spin1))
        self.wait(1)
        self.play(
            particle1.animate.move_to(end_point1),
            run_time=2
        )
        
        # Create and animate vectors
        def create_vector_set(end_point, color_set):
            r_vector = Arrow3D(ORIGIN, end_point, color=color_set['r'])
            p_vector = Arrow3D(end_point, end_point + np.array([1, 0, 0]), color=color_set['p'])
            
            r_vec = end_point - ORIGIN
            p_vec = np.array([1, 0, 0])
            L_vec = np.cross(r_vec, p_vec)
            L_vec = L_vec / np.linalg.norm(L_vec)
            
            L_vector = Arrow3D(end_point, end_point + L_vec, color=color_set['L'])
            return r_vector, p_vector, L_vector
            
        # Vector colors
        colors1 = {'r': RADIUS_COLOR, 'p': MOMENTUM_COLOR, 'L': ANGULAR_MOMENTUM_COLOR}
        vectors1 = create_vector_set(end_point1, colors1)
        
        # Add vector labels with better positioning
        vector_labels = VGroup(
            MathTex("\\vec{r}", color=RADIUS_COLOR),
            MathTex("\\vec{p}", color=MOMENTUM_COLOR),
            MathTex("\\vec{L}", color=ANGULAR_MOMENTUM_COLOR),
        ).arrange(RIGHT, buff=1)
        self.add_fixed_in_frame_mobjects(vector_labels)
        vector_labels.to_corner(DR)

        self.play(
            Create(vectors1[0]),
            Create(vectors1[1]),
            Create(vectors1[2]),
            run_time=2
        )
        self.wait(4)


Manim Community v0.18.1

In [10]:
%%manim -qh NuclearScatteringAsymDownEducational

class NuclearScatteringAsymDownEducational(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        self.add(axes)

        # Create initial demonstration vectors at a fixed point (0, -1, 0)
        fixed_point = np.array([0, -1, 0])
        
        # Momentum vector (along X-axis)
        p_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([1, 0, 0]),
            color=MOMENTUM_COLOR,
            buff=0
        )
        p_label = MathTex("\\vec{p}", color=MOMENTUM_COLOR)
        p_label.next_to(p_vector.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Spin vector (along negative Z-axis for spin down)
        s_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 0, -1]),  # Negative Z for spin down
            color=SPIN_DOWN_COLOR,
            buff=0
        )
        s_label = MathTex("\\vec{S}", color=SPIN_DOWN_COLOR)
        s_label.next_to(s_vector.get_end(), DOWN).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Cross product vector (will appear later)
        cross_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, -1, 0]),  # Negative Y direction
            color=ORANGE,
            buff=0
        )
        cross_label = MathTex("\\vec{p} \\times \\vec{S} = -1", color=ORANGE)
        cross_label.next_to(cross_vector.get_end(), LEFT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        
       
        
        # Now create the orbit animation components
        nucleus = Sphere(radius=0.2, color=GRAY)
        particle = Sphere(radius=0.1, color=SPIN_DOWN_COLOR)
        particle.move_to([0, -1, 0])

        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)


         # Educational sequence
        self.add(axes)

                # Add orbit components
        self.play(
            FadeIn(nucleus),
            FadeIn(particle),
        )
        
        # 1. Show momentum vector
        self.play(
            Create(p_vector),
            Write(p_label)
        )
        self.wait(1)
        
        # 2. Show spin vector
        self.play(
            Create(s_vector),
            Write(s_label)
        )
        self.wait(1)
        
        # 3. Show cross product result
        self.play(
            Create(cross_vector),
            Write(cross_label)
        )
        self.wait(1)

        # 4. Fade out educational vectors and labels
        self.play(
            FadeOut(p_vector),
            FadeOut(s_vector),
            FadeOut(cross_vector),
            FadeOut(p_label),
            FadeOut(s_label),
            FadeOut(cross_label)
        )

        radial_force = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=ORANGE,
            buff=0
        )

        cross_product = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, -1, 0]),
            color=ORANGE,
            buff=0
        )

        def update_radial_force(vector):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            S_cross_p = np.array([0, -1, 0])
            f_radial = np.dot(r, S_cross_p)
            force_vector = f_radial * r_hat
            vector.put_start_and_end_on(ORIGIN, force_vector)

        def update_cross_product(vector):
            pos = particle.get_center()
            vector.put_start_and_end_on(pos, pos + np.array([0, -1, 0]))

        radial_force.add_updater(update_radial_force)
        cross_product.add_updater(update_cross_product)

        self.wait(3)

        # Add orbit components
        self.play(
            Create(orbit_path),
            Create(radial_force),
            Create(cross_product)
        )

        # Add labels for orbital motion
        orbital_label = MathTex("\\hat{r}", "\\vec{r}", "\\cdot", 
            "\\big(", "\\vec{p}", "\\times", "\\vec{S}", "\\big) = ", color=WHITE)
        orbital_label.to_corner(UR).shift(LEFT * 2)
        self.add_fixed_in_frame_mobjects(orbital_label)
        orbital_label[1].set_color(RADIUS_COLOR) # r
        orbital_label[4].set_color(MOMENTUM_COLOR) # p
        orbital_label[5].set_color(SPIN_DOWN_COLOR) # S

        display_text = MathTex("00.00", color=WHITE)
        display_text.next_to(orbital_label, RIGHT, buff=0.32)
        self.add_fixed_in_frame_mobjects(display_text)

        def update_value(mob):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            if r_magnitude > 0:
                r_hat = r / r_magnitude
            else:
                r_hat = np.array([0, 0, 0])
            S_cross_p = np.array([0, -1, 0])
            f_radial = abs(np.dot(r_hat, S_cross_p))
            new_text = MathTex(f"-{f_radial:.2f}", color=FORCE_COLOR)
            new_text.move_to(mob)
            mob.become(new_text)

        display_text.add_updater(update_value)

        for _ in range(4):
        # Perform the orbital rotation
            self.play(
                Rotating(
                    particle,
                    about_point=ORIGIN,
                    axis=RIGHT,
                    angle=TAU,
                    run_time=4,
                    rate_func=linear
                )
            )

        # Clean up
        radial_force.clear_updaters()
        cross_product.clear_updaters()
        display_text.clear_updaters()

Manim Community v0.18.1

In [9]:
%%manim -qh NuclearScatteringAsymRightForces

class NuclearScatteringAsymRightForces(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        self.add(axes)

        # Create initial point and basic components
        fixed_point = np.array([0, -1, 0])
        
        nucleus = Sphere(radius=0.2, color=GRAY)
        particle = Sphere(radius=0.1, color=RED)
        particle.move_to(fixed_point)

        # Add basic components
        self.add(nucleus)
        self.add(particle)
        
        # Create the three demonstration vectors
        p_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([1, 0, 0]),
            color=BLUE,
            buff=0
        )
        p_label = MathTex("\\vec{p}", color=MOMENTUM_COLOR)
        p_label.next_to(p_vector.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        s_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 1, 0]),
            color=GREEN,
            buff=0
        )
        s_label = MathTex("\\vec{S}", color=SPIN_RIGHT_COLOUR)
        s_label.next_to(s_vector.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        cross_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 0, 1]),
            color=ORANGE,
            buff=0
        )
        cross_label = MathTex("\\vec{p} \\times \\vec{S} = +\\hat{z}", color=ORANGE)
        cross_label.next_to(cross_vector.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Create orbit path
        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)

        # Calculate initial radial force
        r = fixed_point
        r_magnitude = np.linalg.norm(r)
        r_hat = r / r_magnitude
        S_cross_p = np.array([0, 0, 1])  # For spin right
        f_radial = np.dot(r, S_cross_p)
        force_vector = f_radial * r_hat

        # Create initial force vectors with correct positions
        radial_force = Arrow(
            start=ORIGIN,
            end=force_vector,
            color=PURPLE,
            buff=0
        )

        cross_product = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 0, 1]),
            color=ORANGE,
            buff=0
        )

        # Show vectors one by one
        self.play(Create(p_vector), Write(p_label))
        self.wait(1)
        
        self.play(Create(s_vector), Write(s_label))
        self.wait(1)
        
        self.play(Create(cross_vector), Write(cross_label))
        self.wait(1)

        # Fade out educational vectors
        self.play(
            FadeOut(p_vector),
            FadeOut(s_vector),
            FadeOut(cross_vector),
            FadeOut(p_label),
            FadeOut(s_label),
            FadeOut(cross_label)
        )

        # Show orbit path and force vectors
        self.play(
            Create(orbit_path),
            Create(radial_force),
            Create(cross_product)
        )
        self.wait(2)

Manim Community v0.18.1

In [74]:
%%manim -qh NuclearScatteringAsymR

class NuclearScatteringAsymR(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        self.add(axes)

        # Create initial demonstration vectors at a fixed point (0, -1, 0)
        fixed_point = np.array([0, -1, 0.01])
        
        # Momentum vector (along X-axis)
        p_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([1, 0, 0]),
            color=BLUE,
            buff=0
        )
        p_label = MathTex("\\vec{p}", color=MOMENTUM_COLOR)
        p_label.next_to(p_vector.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Spin vector (along negative y-axis for spin R)
        s_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, -1, 0]),  # Negative Z for spin R
            color=GREEN,
            buff=0
        )
        s_label = MathTex("\\vec{S}", color=SPIN_LEFT_COLOR)
        s_label.next_to(s_vector.get_end(), DOWN).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Cross product vector (will appear later)
        cross_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 0, -1]),  # Positive Y direction
            color=ORANGE,
            buff=0
        )
        cross_label = MathTex("\\vec{p} \\times \\vec{S} = -y", color=ORANGE)
        cross_label.next_to(cross_vector.get_end(), DOWN).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        
        legend_group = VGroup()

        # Spin right dot and label
        spin_right_dot = Dot(color=SPIN_RIGHT_COLOR)
        spin_right_text = Text("Spin Right", font_size=24, color=SPIN_RIGHT_COLOR).next_to(spin_right_dot, RIGHT)
        spin_right_group = VGroup(spin_right_dot, spin_right_text)

        # Spin left dot and label
        spin_left_dot = Dot(color=SPIN_LEFT_COLOR)
        spin_left_text = Text("Spin Left", font_size=24, color=SPIN_LEFT_COLOR)
        spin_left_text.next_to(spin_left_dot, RIGHT)
        spin_left_group = VGroup(spin_left_dot, spin_left_text)

        # Arrange legend items vertically
        legend_group.add(spin_right_group)
        legend_group.arrange(DOWN, aligned_edge=LEFT, buff=0.3)

        # Add background rectangle
        background = SurroundingRectangle(legend_group, color=WHITE, fill_opacity=0.1, buff=0.2)
        legend_group.add_to_back(background)

        # Position in top right
        legend_group.to_corner(UR, buff=0.5).shift(LEFT*2)



       
        
        # Now create the orbit animation components
        nucleus = Sphere(radius=0.2, color=GRAY)
        particle = Sphere(radius=0.1, color=SPIN_RIGHT_COLOR)
        particle.move_to([0,-1, 0.01])

        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)


         # Educational sequence
        self.add(axes)

        # Add legend to the scene fixed to the frame
        self.add_fixed_in_frame_mobjects(legend_group)

                # Add orbit components
        self.play(
            FadeIn(nucleus),
            FadeIn(particle),
        )
        
        # 1. Show momentum vector
        self.play(
            Create(p_vector),
            Write(p_label)
        )
        self.wait(1)
        
        # 2. Show spin vector
        self.play(
            Create(s_vector),
            Write(s_label)
        )
        self.wait(1)
        
        # 3. Show cross product result
        self.play(
            Create(cross_vector),
            Write(cross_label)
        )
        self.wait(1)

        # 4. Fade out educational vectors and labels
        self.play(
            FadeOut(p_vector),
            FadeOut(s_vector),
            FadeOut(cross_vector),
            FadeOut(p_label),
            FadeOut(s_label),
            FadeOut(cross_label)
        )

        radial_force = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=PURPLE,
            buff=0
        )

        cross_product = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, -1, 0]),
            color=ORANGE,
            buff=0
        )

        def update_radial_force(vector):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            S_cross_p = np.array([0, 0, -1])
            f_radial = np.dot(r, S_cross_p)
            force_vector = f_radial * r_hat
            vector.put_start_and_end_on(ORIGIN, force_vector)

        def update_cross_product(vector):
            pos = particle.get_center()
            vector.put_start_and_end_on(pos, pos + np.array([0, 0, -1]))

        radial_force.add_updater(update_radial_force)
        cross_product.add_updater(update_cross_product)

        self.wait(3)

        # Add orbit components
        self.play(
            Create(orbit_path),
            Create(radial_force),
            Create(cross_product)
        )



        for _ in range(4):
        # Perform the orbital rotation
            self.play(
                Rotating(
                    particle,
                    about_point=ORIGIN,
                    axis=RIGHT,
                    angle=TAU,
                    run_time=4,
                    rate_func=linear
                )
            )

        # Clean up
        radial_force.clear_updaters()
        cross_product.clear_updaters()
        # display_text.clear_updaters()

Manim Community v0.18.1

In [75]:
%%manim -qh NuclearScatteringAsymL

class NuclearScatteringAsymL(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        self.add(axes)

        # Create initial demonstration vectors at a fixed point (0, -1, 0)
        fixed_point = np.array([0, -1, 0.01])
        
        # Momentum vector (along X-axis)
        p_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([1, 0, 0]),
            color=BLUE,
            buff=0
        )
        p_label = MathTex("\\vec{p}", color=MOMENTUM_COLOR)
        p_label.next_to(p_vector.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Spin vector (along positive y-axis for spin L)
        s_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 1, 0]),  # Changed to positive Y for spin L
            color=GREEN,
            buff=0
        )
        s_label = MathTex("\\vec{S}", color=SPIN_LEFT_COLOUR)
        s_label.next_to(s_vector.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Cross product vector (will appear later)
        cross_vector = Arrow(
            start=fixed_point,
            end=fixed_point + np.array([0, 0, 1]),  # Changed to positive Z direction
            color=ORANGE,
            buff=0
        )
        cross_label = MathTex("\\vec{p} \\times \\vec{S} = +y", color=ORANGE)  # Changed label
        cross_label.next_to(cross_vector.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        
        # Now create the orbit animation components
        nucleus = Sphere(radius=0.2, color=GRAY)
        particle = Sphere(radius=0.1, color=SPIN_LEFT_COLOUR)
        particle.move_to([0,-1, 0.01])

        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)

          # Create legend
        legend_group = VGroup()

        # Using softer colors
       # Using softer colors
        LEFT_COLOR = TEAL_B # Terracotta
        RIGHT_COLOR =  DARK_BROWN   # Navy
        
        # Spin right dot and label
        spin_right_dot = Dot(color=SPIN_RIGHT_COLOUR)
        spin_right_text = Text("Spin Right", font_size=24, color=SPIN_RIGHT_COLOUR).next_to(spin_right_dot, RIGHT)
        spin_right_group = VGroup(spin_right_dot, spin_right_text)

        # Spin left dot and label
        spin_left_dot = Dot(color=SPIN_LEFT_COLOUR)
        spin_left_text = Text("Spin Left", font_size=24, color=SPIN_LEFT_COLOUR)
        spin_left_text.next_to(spin_left_dot, RIGHT)
        spin_left_group = VGroup(spin_left_dot, spin_left_text)

        # Arrange legend items vertically
        legend_group.add(spin_left_group)
        legend_group.arrange(DOWN, aligned_edge=LEFT, buff=0.3).shift(LEFT*2)

        # Add background rectangle
        background = SurroundingRectangle(legend_group, color=WHITE, fill_opacity=0.1, buff=0.2)
        legend_group.add_to_back(background)

        # Position in top right
        legend_group.to_corner(UR, buff=0.5).shift(LEFT*2)

        # Educational sequence
        self.add(axes)

        # Add legend to the scene fixed to the frame
        self.add_fixed_in_frame_mobjects(legend_group)

        # Add orbit components
        self.play(
            FadeIn(nucleus),
            FadeIn(particle),
        )
        
        # 1. Show momentum vector
        self.play(
            Create(p_vector),
            Write(p_label)
        )
        self.wait(1)
        
        # 2. Show spin vector
        self.play(
            Create(s_vector),
            Write(s_label)
        )
        self.wait(1)
        
        # 3. Show cross product result
        self.play(
            Create(cross_vector),
            Write(cross_label)
        )
        self.wait(1)

        # 4. Fade out educational vectors and labels
        self.play(
            FadeOut(p_vector),
            FadeOut(s_vector),
            FadeOut(cross_vector),
            FadeOut(p_label),
            FadeOut(s_label),
            FadeOut(cross_label)
        )

        radial_force = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=FORCE_COLOR,
            buff=0
        )

        cross_product = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, 0, 1]),  # Changed to positive Z
            color=ORANGE,
            buff=0
        )

        def update_radial_force(vector):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            S_cross_p = np.array([0, 0, 1])  # Changed to positive Z
            f_radial = np.dot(r, S_cross_p)
            force_vector = f_radial * r_hat
            vector.put_start_and_end_on(ORIGIN, force_vector)

        def update_cross_product(vector):
            pos = particle.get_center()
            vector.put_start_and_end_on(pos, pos + np.array([0, 0, 1]))  # Changed to positive Z

        radial_force.add_updater(update_radial_force)
        cross_product.add_updater(update_cross_product)

        self.wait(3)

        # Add orbit components
        self.play(
            Create(orbit_path),
            Create(radial_force),
            Create(cross_product)
        )

      


        for _ in range(4):
            # Perform the orbital rotation
            self.play(
                Rotating(
                    particle,
                    about_point=ORIGIN,
                    axis=RIGHT,
                    angle=TAU,
                    run_time=4,
                    rate_func=linear
                )
            )

        # Clean up
        radial_force.clear_updaters()
        cross_product.clear_updaters()

Manim Community v0.18.1

In [85]:
%%manim -qh NuclearScatteringLongitudinal

class NuclearScatteringLongitudinal(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)
        self.add(axes)

        # Create initial point and basic components
        fixed_point = np.array([0, -1, 0.01])
        
        # Create basic components
        nucleus = Sphere(radius=0.2, color=GRAY)
        particle = Sphere(radius=0.1, color=PINK)
        particle.move_to(fixed_point)

        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)

        # Create and add initial vectors
        p_vector = Arrow(
            start=particle.get_center() + np.array([0, 0, 0.04]),
            end=particle.get_center() + np.array([0.8, 0, 0.04]),
            color=MOMENTUM_COLOR,
            buff=0
        )
        s_vector = Arrow(
            start=particle.get_center() + np.array([0, 0,-0.02]),
            end=particle.get_center() + np.array([1.2, 0,-0.02]),
            color=PINK,
            buff=0
        )

               # r-vector: points from nucleus (origin) to particle
        r_vector = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=YELLOW,
            buff=0
        )

                # Updater for r-vector
        def update_r(vector):
            vector.put_start_and_end_on(ORIGIN, particle.get_center())

        # Updater for p-vector (along X-axis)
        def update_p(vector):
            pos = particle.get_center()
            vector.put_start_and_end_on(pos, pos + np.array([1, 0, 0]))

        # Add explanation text
        formula = MathTex("\\vec{r}", "\\cdot",  "(\\vec{p}", "\\times", "\\vec{S})", "= 0", color=WHITE)
        formula.to_corner(UR)
        formula[0].set_color(YELLOW)
        formula[2].set_color(MOMENTUM_COLOR)
        formula[4].set_color(PINK)

        self.add_fixed_in_frame_mobjects(formula)


        # Add components to scene
        self.add(axes, nucleus, orbit_path)
        self.play(FadeIn(particle), Create(p_vector), Create(s_vector), Create(r_vector))

        def update_vectors(mobj, dt):
            p_vector.put_start_and_end_on(
                particle.get_center() + np.array([0, 0, 0.04]),
                particle.get_center() + np.array([0.8, 0, 0.04]),
            )
            s_vector.put_start_and_end_on(
                particle.get_center() + np.array([0, 0, -0.02]),
                particle.get_center() + np.array([1.2, 0, -0.02]),
            )

        particle.add_updater(update_vectors)
        r_vector.add_updater(update_r)


        # Perform the orbital rotation
        for _ in range(4):
            self.play(
                Rotating(
                    particle,
                    about_point=ORIGIN,
                    axis=RIGHT,
                    angle=TAU,
                    run_time=4,
                    rate_func=linear
                )
            )

        # Clean up
        particle.clear_updaters()

Manim Community v0.18.1